In [2]:
import os
import pandas as pd
path_to_incubations = "../../lab_work/sessions/incubations"


def load_incubation_data(path_to_incubations):
    dfs = []
    for file_name in os.listdir(path_to_incubations):
        path_to_file = os.path.join(path_to_incubations, file_name)
        print(path_to_file)
        dfs.append(pd.read_csv(path_to_file))

    result = pd.concat(dfs, ignore_index=True)

    return result
    # print(path_to_file)
    
def tidy_incubation_data(df):
    df['sample_name'] = df['sample_id']
    df['sample_id'] = df.apply(lambda row: set_sample_id(row), axis=1)
    df = df.drop(['sample_name'], axis=1)
    return df
    



def set_sample_id(row):
    sample_name = row['sample_name']
    if sample_name.startswith('40ml'):
        return f"40mL_{sample_name[-3:]}"
    else:
        return f"2mL_{sample_name}"
        
incubation_df = load_incubation_data(path_to_incubations=path_to_incubations)
incubation_df = tidy_incubation_data(incubation_df)
incubation_df

../../lab_work/sessions/incubations/2mLWetSpirullina.csv
../../lab_work/sessions/incubations/40mLDrySpirullina.csv
../../lab_work/sessions/incubations/2mLDrySpirullina.csv
../../lab_work/sessions/incubations/40mLWetSpirullina.csv


,sample_id,ratio,intended weight (mg),weight (mg),notes,incubation_start_date,biomass (mg),salt (mg),water (mg),innoculum added
0,2mL_2.1.0,01:00,109,129.0,NaN,7/12/23,NaN,NaN,NaN,NaN
1,2mL_2.1.1,01:00,109,119.0,NaN,7/12/23,NaN,NaN,NaN,NaN
2,2mL_2.1.2,01:00,109,165.0,NaN,7/12/23,NaN,NaN,NaN,NaN
3,2mL_2.2.0,01:01,215,350.0,NaN,7/12/23,NaN,NaN,NaN,NaN
4,2mL_2.2.1,01:01,215,248.0,NaN,7/12/23,NaN,NaN,NaN,NaN
5,2mL_2.2.2,01:01,215,241.0,NaN,7/12/23,NaN,NaN,NaN,NaN
6,2mL_2.3.0,16:01,115,118.0,NaN,7/12/23,NaN,NaN,NaN,NaN
7,2mL_2.3.1,16:01,115,121.0,NaN,7/12/23,NaN,NaN,NaN,NaN
8,2mL_2.3.2,16:01,115,124.0,NaN,7/12/23,NaN,NaN,NaN,NaN
9,2mL_2.4.0,01:05,635,707.0,NaN,7/12/23,NaN,NaN,NaN,NaN


In [3]:
import numpy as np
data_df = pd.read_csv('../../GCTCD_PDF_READER/GCPDFReader/csvs/tidied_1692821404.csv')

# np.unique(data_df['sample_id'])
data_df

,Unnamed: 0.1,Unnamed: 0,Sample_Name,Sample_Date,Instrument,Peak,Time,Type,Area,Height,Width,Start,End,pdf_file_name,sample_id,peak_compound,is_std,known_conc,calculated_conc
0,0,0,40ML1_10,8/16/2023,GCTCD,1,3.151,BP,494087.00000,188091.00000,0.0386,3.107,3.267,../../lab_work/sessions/GCTCD/20230816/40mL_1....,40mL_1.1,NaN,False,NaN,NaN
1,1,1,40ML1_10,8/16/2023,GCTCD,2,4.693,PP,1595.22729,652.50348,0.0375,4.642,4.791,../../lab_work/sessions/GCTCD/20230816/40mL_1....,40mL_1.1,CO2,False,NaN,934.370010
2,2,2,40ML1_10,8/16/2023,GCTCD,3,5.270,BP,124.49847,48.04475,0.0332,5.238,5.311,../../lab_work/sessions/GCTCD/20230816/40mL_1....,40mL_1.1,NaN,False,NaN,NaN
3,3,3,40ML1_10,8/16/2023,GCTCD,4,5.853,PV,188.87746,73.72960,0.0339,5.817,5.888,../../lab_work/sessions/GCTCD/20230816/40mL_1....,40mL_1.1,NaN,False,NaN,NaN
4,4,4,40ML1_10,8/16/2023,GCTCD,5,5.961,VV,529.92957,109.93177,0.0602,5.888,6.011,../../lab_work/sessions/GCTCD/20230816/40mL_1....,40mL_1.1,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,424,35,2.979pp m,8/14/2023,GCFID,1,2.387,MF,240.27603,31.48297,0.1272,0.000,2.495,../../lab_work/sessions/GCFID/20230814/2023081...,CO2_600ppm_CH4_2179ppb,CO,True,2.031,NaN
425,425,36,2.979pp m,8/14/2023,GCFID,2,2.495,FM,7.25621,4.14673,0.0292,2.495,2.521,../../lab_work/sessions/GCFID/20230814/2023081...,CO2_600ppm_CH4_2179ppb,NaN,True,NaN,NaN
426,426,37,2.979pp m,8/14/2023,GCFID,3,2.567,FM,185.20935,15.03130,0.2054,2.521,4.998,../../lab_work/sessions/GCFID/20230814/2023081...,CO2_600ppm_CH4_2179ppb,CH4,True,2.979,178.510783
427,427,38,2.979pp m,8/14/2023,GCFID,1,2.385,BV,120.97746,26.07795,0.0561,2.310,2.493,../../lab_work/sessions/GCFID/20230814/2023081...,CO2_600ppm_CH4_2179ppb,CO,True,2.031,NaN


In [4]:
new_df = pd.merge(data_df, incubation_df, on='sample_id', how='left')
# new_df[new_df['Sample_Name']=='2ML_2_10']['sample_id']
new_df

,Unnamed: 0.1,Unnamed: 0,Sample_Name,Sample_Date,Instrument,Peak,Time,Type,Area,Height,...,calculated_conc,ratio,intended weight (mg),weight (mg),notes,incubation_start_date,biomass (mg),salt (mg),water (mg),innoculum added
0,0,0,40ML1_10,8/16/2023,GCTCD,1,3.151,BP,494087.00000,188091.00000,...,NaN,01:00,1750.0,NaN,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley
1,1,1,40ML1_10,8/16/2023,GCTCD,2,4.693,PP,1595.22729,652.50348,...,934.370010,01:00,1750.0,NaN,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley
2,2,2,40ML1_10,8/16/2023,GCTCD,3,5.270,BP,124.49847,48.04475,...,NaN,01:00,1750.0,NaN,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley
3,3,3,40ML1_10,8/16/2023,GCTCD,4,5.853,PV,188.87746,73.72960,...,NaN,01:00,1750.0,NaN,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley
4,4,4,40ML1_10,8/16/2023,GCTCD,5,5.961,VV,529.92957,109.93177,...,NaN,01:00,1750.0,NaN,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,424,35,2.979pp m,8/14/2023,GCFID,1,2.387,MF,240.27603,31.48297,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425,425,36,2.979pp m,8/14/2023,GCFID,2,2.495,FM,7.25621,4.14673,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,426,37,2.979pp m,8/14/2023,GCFID,3,2.567,FM,185.20935,15.03130,...,178.510783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427,427,38,2.979pp m,8/14/2023,GCFID,1,2.385,BV,120.97746,26.07795,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
np.unique(new_df[new_df['incubation_start_date'].isnull()]['sample_id'])

array(['AMBIENT_AIR', 'CH4_2pph', 'CO2_20pph', 'CO2_600ppm_CH4_2179ppb',
       'DROP_ME'], dtype=object)

In [6]:

from datetime import datetime

def set_incubation_length(row):
    try:
        sample_date  = datetime.strptime(row['Sample_Date'], '%m/%d/%Y')
        incubation_start_date  = datetime.strptime(row['incubation_start_date'], '%m/%d/%y')
        return sample_date - incubation_start_date

    except Exception as e:
        print(e, row['Sample_Date'], row['incubation_start_date'])
        return None
new_df['incubation_length'] = new_df.apply(lambda row: set_incubation_length(row), axis=1)

new_df

strptime() argument 1 must be str, not float 7/23/2023 nan
strptime() argument 1 must be str, not float 7/23/2023 nan
strptime() argument 1 must be str, not float 7/23/2023 nan
strptime() argument 1 must be str, not float 7/23/2023 nan
strptime() argument 1 must be str, not float 7/23/2023 nan
strptime() argument 1 must be str, not float 7/23/2023 nan
strptime() argument 1 must be str, not float 7/23/2023 nan
strptime() argument 1 must be str, not float 7/23/2023 nan
strptime() argument 1 must be str, not float 7/23/2023 nan
strptime() argument 1 must be str, not float 7/23/2023 nan
strptime() argument 1 must be str, not float 7/23/2023 nan
strptime() argument 1 must be str, not float 7/23/2023 nan
strptime() argument 1 must be str, not float 7/23/2023 nan
strptime() argument 1 must be str, not float 7/23/2023 nan
strptime() argument 1 must be str, not float 7/23/2023 nan
strptime() argument 1 must be str, not float 7/23/2023 nan
strptime() argument 1 must be str, not float 7/23/2023 n

,Unnamed: 0.1,Unnamed: 0,Sample_Name,Sample_Date,Instrument,Peak,Time,Type,Area,Height,...,ratio,intended weight (mg),weight (mg),notes,incubation_start_date,biomass (mg),salt (mg),water (mg),innoculum added,incubation_length
0,0,0,40ML1_10,8/16/2023,GCTCD,1,3.151,BP,494087.00000,188091.00000,...,01:00,1750.0,NaN,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley,51 days
1,1,1,40ML1_10,8/16/2023,GCTCD,2,4.693,PP,1595.22729,652.50348,...,01:00,1750.0,NaN,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley,51 days
2,2,2,40ML1_10,8/16/2023,GCTCD,3,5.270,BP,124.49847,48.04475,...,01:00,1750.0,NaN,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley,51 days
3,3,3,40ML1_10,8/16/2023,GCTCD,4,5.853,PV,188.87746,73.72960,...,01:00,1750.0,NaN,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley,51 days
4,4,4,40ML1_10,8/16/2023,GCTCD,5,5.961,VV,529.92957,109.93177,...,01:00,1750.0,NaN,NaN,6/26/23,1750.0,0.0,92.105263,~100 mg sand from death valley,51 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,424,35,2.979pp m,8/14/2023,GCFID,1,2.387,MF,240.27603,31.48297,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
425,425,36,2.979pp m,8/14/2023,GCFID,2,2.495,FM,7.25621,4.14673,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
426,426,37,2.979pp m,8/14/2023,GCFID,3,2.567,FM,185.20935,15.03130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
427,427,38,2.979pp m,8/14/2023,GCFID,1,2.385,BV,120.97746,26.07795,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [7]:
relevant_df  = new_df[(new_df['peak_compound'].notnull()) & (new_df['sample_id']!="DROP_ME")]
methane_df = relevant_df[(relevant_df['peak_compound'] == 'CH4') & (relevant_df['is_std'] == False)]
CO2_df = relevant_df[(relevant_df['peak_compound'] == 'CO2') & (relevant_df['is_std'] == False)]


In [ ]:
CO2_df.columns

In [10]:
import bokeh
from bokeh.plotting import figure, show
# bokeh.io.show()

p = figure(width=400, height=400)
p.circle('calculated_conc','incubation_length',source=CO2_df ,fill_alpha=0.2, size=10)
bokeh.io.show(p)
